In [4]:
#nltk.download('punkt')
#!pip install nltk
#nltk.download('stopwords')

In [6]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import string
import time

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from numpy import savetxt

In [7]:
# Input - Utterance or Column '1'
# Emotion(7) - Output 1 and Column '3'
# Sentiment(3) - Output 2 and Column '4'
df1 = pd.read_csv('data1.csv')
df2 = pd.read_csv('data2.csv')
df3 = pd.read_csv('data3.csv')
df4 = pd.read_csv('data4.csv')

frames = [df1,df2,df3,df4]
data = pd.concat(frames) #complete dataset

mat =np.asarray([]) #emo
mat1 =np.asarray([]) #senti
data = np.asarray(data[['1','3','4']])

In [8]:
vect = CountVectorizer()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data[:,0], data[:,1:3], test_size=0.2, shuffle = True)

X_train_copy1 = X_train.copy()
X_test_copy1 = X_test.copy()

X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

# emotion (7)
y_train_emo = y_train[:,0] *1.0
y_train_emo = y_train_emo.astype('int')
y_test_emo = y_test[:,0] *1.0
y_test_emo = y_test_emo.astype('int')

# sentiment(3)
y_train_senti = y_train[:,1] *1.0
y_train_senti = y_train_senti.astype('int')
y_test_senti = y_test[:,1] *1.0
y_test_senti = y_test_senti.astype('int')

# ------------------------------------------------------------------------------------------------------

# Multinomial Logistic Regression

In [9]:
def logistic_regression(X_train, y_train, X_test, y_test):
    print("Multinomial Logistic Regression")
    start = time.time()
    logreg = LogisticRegression(multi_class='multinomial', max_iter = 1e4)
    logreg.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(logreg, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    accuracy_test_data = cross_val_score(logreg, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    y_pred = logreg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.5163227572939671
Testing Accuracy :  0.4759330168986745
[[1089   35    3   20   77    5   57]
 [ 183  104    2    6   22    2   11]
 [  43    6    4    7    4    0    6]
 [ 124   10    4   20   21    3   19]
 [ 282   26    2    9  116    1   22]
 [  47    6    0    4    6    4   14]
 [ 207   25    2    8   23    2   49]]
F1-Score :  0.5054704595185996
Time Taken : 11.005656719207764

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.5534380337537061
Testing Accuracy :  0.5069356361267334
[[959 116 211]
 [299 193 115]
 [427  72 350]]
F1-Score :  0.5477753464624362
Time Taken : 5.83949875831604


# Naive Bayes

In [10]:
def naive_bayes(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Multinomial Naive Bayes")
    classifier = MultinomialNB();
    classifier.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(classifier, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = classifier.predict(X_test)
    accuracy_test_data = cross_val_score(classifier, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.5008210844067121
Testing Accuracy :  0.482132743009852
[[1195   10    1    1   48    0   31]
 [ 226   77    0    0   21    0    6]
 [  59    3    0    2    2    0    4]
 [ 165    7    0    6   16    0    7]
 [ 346   17    0    2   83    0   10]
 [  65    1    0    2    3    1    9]
 [ 255   15    0    2   14    0   30]]
F1-Score :  0.5076586433260394
Time Taken : 0.04716897010803223

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.5538030802408093
Testing Accuracy :  0.5109482403307939
[[1015   72  199]
 [ 319  162  126]
 [ 438   54  357]]
F1-Score :  0.5594456601021153
Time Taken : 0.024672269821166992


In [11]:
mat.reshape(2,6)

array([[5.16322757e-01, 1.69595982e-02, 4.75933017e-01, 1.33925514e-02,
        5.05470460e-01, 1.10056567e+01],
       [5.00821084e-01, 5.06851321e-03, 4.82132743e-01, 6.26946602e-03,
        5.07658643e-01, 4.71689701e-02]])

# Random Forest

In [12]:
def random_forest(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Random Forest Classifier")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(rf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))

    # predicting test set results
    y_pred = rf.predict(X_test)
    accuracy_test_data = cross_val_score(rf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start



print("FOR EMOTIONS (7): ")
tup = random_forest(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup= random_forest(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.5095752998789136
Testing Accuracy :  0.48541010197705187
[[1166   45    2    6   40    0   27]
 [ 174  124    4    2   14    2   10]
 [  57    2    4    3    2    0    2]
 [ 155   17    2   13    7    0    7]
 [ 326   28    3    1   84    0   16]
 [  65    5    0    1    4    1    5]
 [ 243   28    0    2   11    0   32]]
F1-Score :  0.5193289569657185
Time Taken : 89.89220762252808

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.5475104552855463
Testing Accuracy :  0.5124021113371359
[[1025   92  169]
 [ 322  179  106]
 [ 471   52  326]]
F1-Score :  0.5579868708971554
Time Taken : 84.55184435844421


# Support Vector Classifier

In [13]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4906988383108432
Testing Accuracy :  0.4690000398867217
[[1269    2    0    1    8    0    6]
 [ 294   30    0    0    5    0    1]
 [  63    1    0    2    2    0    2]
 [ 191    7    0    1    2    0    0]
 [ 406    6    0    0   39    1    6]
 [  78    1    0    0    1    0    1]
 [ 297   10    0    1    3    0    5]]
F1-Score :  0.49015317286652077
Time Taken : 71.88595509529114

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5205189213545545
Testing Accuracy :  0.4726476805871325
[[1200   18   68]
 [ 480   69   58]
 [ 664   20  165]]
F1-Score :  0.5229759299781181
Time Taken : 49.92970061302185


# ----------------------------------------------------------------------------------------------------------

# Feature Engineering - 1. Removing Punctuations

In [14]:
def punctuation_remover(x):
    X_t1 = x
    for i in range(len(x)):
        X_t1[i] = [char for char in x[i] if char not in string.punctuation]
        X_t1[i]=''.join(X_t1[i])
    return X_t1

#All useless punctuations are removed
X_tr1 = punctuation_remover(X_train_copy1)
X_te1 = punctuation_remover(X_test_copy1)

# train1 and test1 - after punctuations removed 
X_train1 = vect.fit_transform(X_tr1) ##fit and transform at the same time
X_test1 = vect.transform(X_te1) ##only transform the test set so that fitting can be tested

# After 1st Feature Engineering 

In [15]:
# Multinomial Logistic Regression

In [16]:
print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.5161398183746162
Testing Accuracy :  0.4741141823886828
[[1078   39    5   25   83    4   52]
 [ 167  124    1    5   19    2   12]
 [  44    7    3    7    3    0    6]
 [ 127   10    4   18   21    3   18]
 [ 280   25    2    9  120    1   21]
 [  45    7    0    6    6    3   14]
 [ 210   21    3   10   24    2   46]]
F1-Score :  0.5076586433260394
Time Taken : 14.585546493530273

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.5506103991277458
Testing Accuracy :  0.5018294709691145
[[966 120 200]
 [309 186 112]
 [423  83 343]]
F1-Score :  0.5452224653537564
Time Taken : 5.946001052856445


In [17]:
# Naive Bayes 

In [18]:
print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.49571413310188495
Testing Accuracy :  0.4795793280416949
[[1193   10    1    3   48    0   31]
 [ 226   78    0    2   18    0    6]
 [  61    2    0    1    2    0    4]
 [ 169    5    0    7   14    0    6]
 [ 355   13    0    3   78    0    9]
 [  70    1    0    2    1    0    7]
 [ 254   13    0    3   15    0   31]]
F1-Score :  0.5058351568198395
Time Taken : 0.06742572784423828

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.5496082463427804
Testing Accuracy :  0.5054811003416961
[[1025   70  191]
 [ 330  150  127]
 [ 446   45  358]]
F1-Score :  0.5590809628008753
Time Taken : 0.04668092727661133


In [19]:
# Random Forest 

In [20]:
print("FOR EMOTIONS (7): ")
tup = random_forest(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = random_forest(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.5103952203933873
Testing Accuracy :  0.476293991730153
[[1147   49    1   13   43    0   33]
 [ 181  118    3    3   13    2   10]
 [  55    4    3    3    3    0    2]
 [ 160   16    0   12    7    1    5]
 [ 337   28    2    0   78    0   13]
 [  68    4    0    1    3    1    4]
 [ 245   27    0    1    9    0   34]]
F1-Score :  0.5080233406272794
Time Taken : 95.37261414527893

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.5421297814667619
Testing Accuracy :  0.506564024836132
[[1031   84  171]
 [ 330  176  101]
 [ 481   48  320]]
F1-Score :  0.5568927789934355
Time Taken : 86.31191730499268


In [21]:
# Support Vector Classifier

In [22]:
print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4886924959294947
Testing Accuracy :  0.4690000398867217
[[1268    1    0    1   11    0    5]
 [ 292   34    0    0    3    0    1]
 [  63    1    0    2    2    0    2]
 [ 191    6    0    1    3    0    0]
 [ 411    6    0    0   34    1    6]
 [  75    1    0    0    1    2    2]
 [ 301    6    0    1    2    1    5]]
F1-Score :  0.49015317286652077
Time Taken : 64.53780722618103

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5171448393225981
Testing Accuracy :  0.47155544919096437
[[1199   18   69]
 [ 475   65   67]
 [ 670   22  157]]
F1-Score :  0.5182348650619986
Time Taken : 45.721609354019165


# Feature Engineering - 2. Removing Stopwords

In [23]:
StopWords = set(stopwords.words('english'))
#print(StopWords)

#train2 and test2 after stopwords removed 
def stopwords_remover(x):
    for i in range(len(x)):
        x[i] = [word for word in x[i].split() if word.lower() not in StopWords]
        
    a = ""
    for i in range(len(x)):
        for word in x[i]:
            #print(word)
            a = a +word +" "
        x[i]=a
        a=""
    return x

X_tr2 = stopwords_remover(X_tr1.copy())
X_te2 = stopwords_remover(X_te1.copy())

#print (X_tr2)
X_train2 = vect.fit_transform(X_tr2.copy()) ##fit and transform at the same time
X_test2 = vect.transform(X_te2.copy()) ##only transform the test set so that fitting can be tested

# After 2nd Feature Engineering 

In [24]:
# Multinomial Logistic Regression

In [25]:
print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train2, y_train_emo, X_test2, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train2, y_train_senti, X_test2, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.5069303131992446
Testing Accuracy :  0.4744778163349421
[[1099   31    3   19   77    3   54]
 [ 203   82    1    6   21    1   16]
 [  47    5    2    7    5    0    4]
 [ 124   11    7   19   18    3   19]
 [ 294   28    0    9  109    0   18]
 [  48    7    0    4   11    2    9]
 [ 214   19    1   11   22    0   49]]
F1-Score :  0.49671772428884026
Time Taken : 9.778442621231079

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.5448653439031375
Testing Accuracy :  0.49271269594352046
[[979 104 203]
 [327 174 106]
 [448  85 316]]
F1-Score :  0.5357403355215171
Time Taken : 4.8530964851379395


In [26]:
# Naive Bayes

In [27]:
print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train2, y_train_emo, X_test2, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)

print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train2, y_train_senti, X_test2, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.4910635522573067
Testing Accuracy :  0.4828553574515044
[[1187   10    0    3   53    0   33]
 [ 243   56    1    1   20    0    9]
 [  58    4    0    2    4    0    2]
 [ 165   11    0    6   13    0    6]
 [ 352   12    0    3   81    0   10]
 [  62    2    0    3    5    1    8]
 [ 249   15    0    3   20    0   29]]
F1-Score :  0.4959883296863603
Time Taken : 0.048004865646362305

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.5353822409165485
Testing Accuracy :  0.5025560740829378
[[1017   82  187]
 [ 344  152  111]
 [ 458   67  324]]
F1-Score :  0.5444930707512764
Time Taken : 0.03200340270996094


In [28]:
# Random Forest

In [29]:
print("FOR EMOTIONS (7): ")
tup = random_forest(X_train2, y_train_emo, X_test2, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = random_forest(X_train2, y_train_senti, X_test2, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.4720973462841285
Testing Accuracy :  0.4445760706260885
[[1014   76    4   34   89    4   65]
 [ 180   91    2    5   26    3   23]
 [  39    9    3    6    8    1    4]
 [ 117   24    0   24   20    2   14]
 [ 294   29    2   10   99    0   24]
 [  53    5    0    6    6    3    8]
 [ 182   35    2    8   34    1   54]]
F1-Score :  0.4697301239970824
Time Taken : 104.84589457511902

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.5183305129730339
Testing Accuracy :  0.48397019132330843
[[947 148 191]
 [298 199 110]
 [447 103 299]]
F1-Score :  0.5269876002917578
Time Taken : 97.47548651695251


In [30]:
# Support Vector Classifier

In [31]:
print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train2, y_train_emo, X_test2, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train2, y_train_senti, X_test2, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4870510753325094
Testing Accuracy :  0.46936500339037135
[[1265    1    0    1   13    0    6]
 [ 295   30    0    0    4    0    1]
 [  63    1    0    2    2    0    2]
 [ 190    6    0    1    3    0    1]
 [ 416    4    0    0   31    1    6]
 [  75    1    0    0    2    1    2]
 [ 300    7    0    1    3    1    4]]
F1-Score :  0.48577680525164113
Time Taken : 44.66665840148926

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5142265458047712
Testing Accuracy :  0.4697293021153259
[[1206   19   61]
 [ 487   60   60]
 [ 678   20  151]]
F1-Score :  0.5167760758570387
Time Taken : 31.804723978042603


In [32]:
mat_emo = np.reshape(mat,(12,6))
mat_senti = np.reshape(mat1,(12,6))

In [33]:
# after lemma - we become 16 by 6

# Feature Engineering - 3. Stemming

In [34]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

porter_stemmer = PorterStemmer()

def stem_sentences(x):
    sen_list = []
    for sentence in x:
        tokens = sentence.split()
        stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
        stemmed_string =  ' '.join(stemmed_tokens)
        sen_list.append(stemmed_string)
    sen_list_arr = np.asarray(sen_list)
    return sen_list_arr
        

#Stemming the sentences
X_tr3 = stem_sentences(X_tr2.copy())
X_te3 = stem_sentences(X_te2.copy())

#Train3 and test3 after stemming is done
X_train3 = vect.fit_transform(X_tr3.copy()) ##fit and transform at the same time
X_test3 = vect.transform(X_te3.copy()) ##only transform the test set so that fitting can be tested

# After 3rd Feature Engineering

In [35]:
# Multinomial Logistic Regression

In [36]:
print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train3, y_train_emo, X_test3, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train3, y_train_senti, X_test3, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.5043770246009409
Testing Accuracy :  0.4708261869623603
[[1099   28    4   28   77    1   49]
 [ 203   84    1    6   22    0   14]
 [  49    4    1    9    3    0    4]
 [ 124   13    6   14   18    4   22]
 [ 296   21    1    9  112    0   19]
 [  48    7    0    6   10    3    7]
 [ 213   18    2    9   25    0   49]]
F1-Score :  0.49671772428884026
Time Taken : 10.94667911529541

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.5434073194688827
Testing Accuracy :  0.49489582917846653
[[988 107 191]
 [326 176 105]
 [445  85 319]]
F1-Score :  0.5408460977388767
Time Taken : 4.382315158843994


In [37]:
# Naive Bayes

In [38]:
print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train3, y_train_emo, X_test3, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)

print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train3, y_train_senti, X_test3, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.4930701856117148
Testing Accuracy :  0.47957733370560945
[[1180   14    0    8   54    0   30]
 [ 237   59    0    1   19    0   14]
 [  57    4    1    2    4    0    2]
 [ 167    9    0    4   14    0    7]
 [ 350   13    0    1   84    0   10]
 [  57    4    0    4    8    1    7]
 [ 239   16    0    5   22    0   34]]
F1-Score :  0.49708242159008026
Time Taken : 0.04028892517089844

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.5420396213858548
Testing Accuracy :  0.5007206201055668
[[1016   86  184]
 [ 346  148  113]
 [ 438   68  343]]
F1-Score :  0.549598832968636
Time Taken : 0.024397611618041992


In [39]:
# Random Forest

In [40]:
print("FOR EMOTIONS (7): ")
tup = random_forest(X_train3, y_train_emo, X_test3, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = random_forest(X_train3, y_train_senti, X_test3, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.47227883033818135
Testing Accuracy :  0.44530865674816855
[[1018   79    2   29   90    4   64]
 [ 183   86    2    4   26    4   25]
 [  42   10    3    5    6    1    3]
 [ 118   28    0   22   19    1   13]
 [ 283   35    2    9  101    1   27]
 [  48    6    0    7   10    4    6]
 [ 174   41    1    9   33    2   56]]
F1-Score :  0.47045951859956237
Time Taken : 90.59397435188293

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.5212480582744217
Testing Accuracy :  0.47959395317298875
[[923 166 197]
 [298 203 106]
 [427 116 306]]
F1-Score :  0.5222465353756383
Time Taken : 84.82450985908508


In [41]:
# Support Vector Classifier

In [42]:
print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train3, y_train_emo, X_test3, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train3, y_train_senti, X_test3, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.48686888462959754
Testing Accuracy :  0.4682707776581176
[[1262    1    0    3   17    0    3]
 [ 285   35    0    0    8    0    2]
 [  66    1    0    2    1    0    0]
 [ 188    6    0    2    4    0    1]
 [ 410    7    0    0   35    1    5]
 [  74    1    0    0    2    1    3]
 [ 288    9    0    3    7    1    8]]
F1-Score :  0.4897884755652808
Time Taken : 43.8143093585968

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5159593319424821
Testing Accuracy :  0.471916424022443
[[1181   24   81]
 [ 485   63   59]
 [ 651   21  177]]
F1-Score :  0.5182348650619986
Time Taken : 31.48685574531555


In [43]:
mat_emo = np.reshape(mat,(16,6))
mat_senti = np.reshape(mat1,(16,6))

In [44]:
mat_emo

array([[5.16322757e-01, 1.69595982e-02, 4.75933017e-01, 1.33925514e-02,
        5.05470460e-01, 1.10056567e+01],
       [5.00821084e-01, 5.06851321e-03, 4.82132743e-01, 6.26946602e-03,
        5.07658643e-01, 4.71689701e-02],
       [5.09575300e-01, 3.52672472e-03, 4.85410102e-01, 1.21348397e-02,
        5.19328957e-01, 8.98922076e+01],
       [4.90698838e-01, 3.96484120e-03, 4.69000040e-01, 1.01967734e-03,
        4.90153173e-01, 7.18859551e+01],
       [5.16139818e-01, 1.46074519e-02, 4.74114182e-01, 1.42242029e-02,
        5.07658643e-01, 1.45855465e+01],
       [4.95714133e-01, 3.01903737e-03, 4.79579328e-01, 5.71497242e-03,
        5.05835157e-01, 8.37564468e-02],
       [5.10395220e-01, 4.49274225e-03, 4.76293992e-01, 1.52347223e-02,
        5.08023341e-01, 9.53726141e+01],
       [4.88692496e-01, 3.80798788e-03, 4.69000040e-01, 1.01967734e-03,
        4.90153173e-01, 6.45378072e+01],
       [5.06930313e-01, 1.09699352e-02, 4.74477816e-01, 1.94884836e-02,
        4.96717724e-01, 

In [48]:
mat_senti[:,:3]

array([[0.55343803, 0.01058488, 0.50693564],
       [0.55380308, 0.0039437 , 0.51094824],
       [0.54751046, 0.0046041 , 0.51240211],
       [0.52051892, 0.00614652, 0.47264768],
       [0.5506104 , 0.01333102, 0.50182947],
       [0.54960825, 0.00476774, 0.5054811 ],
       [0.54212978, 0.01137655, 0.50656402],
       [0.51714484, 0.00619722, 0.47155545],
       [0.54486534, 0.00745299, 0.4927127 ],
       [0.53538224, 0.00415154, 0.50255607],
       [0.51833051, 0.00839856, 0.48397019],
       [0.51422655, 0.00594202, 0.4697293 ],
       [0.54340732, 0.00767265, 0.49489583],
       [0.54203962, 0.00399517, 0.50072062],
       [0.52124806, 0.00516952, 0.47959395],
       [0.51595933, 0.0058084 , 0.47191642]])

In [51]:
# save to csv file
savetxt('mat_senti.csv', mat_senti, delimiter=',')

In [52]:
# save to csv file
savetxt('mat_emo.csv', mat_emo, delimiter=',')